In [1]:
from core import Seed, init

init()

In [2]:
# load all the companies from Airtable
import airtable

companies = airtable.load_into_pandas()
companies.head()

,id,createdTime,fields.extra_json,fields.Name,fields.Website,fields.Status,fields.Created,fields.Last Modified,fields.Created By.id,fields.Created By.email,fields.Created By.name,fields.Last Modified By.id,fields.Last Modified By.email,fields.Last Modified By.name,fields.Key Product Name,fields.Keywords,domain
3,rec5nPLxKw5vzPAgP,2024-10-02T16:56:55.000Z,"{""name"":""TBD"",""description"":""TBD is Block\u201...",TBD,https://tbd.website/,Approved,2024-10-02T16:56:55.000Z,2024-10-02T20:34:40.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,tbd.website
5,rec7STu2pyWvo9cIp,2024-10-02T16:56:58.000Z,"{""name"":""Veritone"",""description"":""Veritone has...",Veritone,http://www.veritone.com/,Approved,2024-10-02T16:56:58.000Z,2024-10-07T21:34:58.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,veritone.com
6,rec7Xm4DXzYQ9uoeb,2024-10-02T16:56:56.000Z,"{""name"":""DoubleDown Interactive LLC"",""descript...",DoubleDown Interactive,https://www.doubledowninteractive.com/jobs/,Approved,2024-10-02T16:56:56.000Z,2024-10-04T19:39:49.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,doubledowninteractive.com
7,rec7zrOPWtNbHBjUp,2024-10-08T18:11:30.000Z,NaN,Sourcegraph,https://sourcegraph.com/,Approved,2024-10-08T18:11:30.000Z,2024-10-10T15:02:54.000Z,usrPAGESHARE00000,anonymous+formpage@noreply.airtable.com,Anonymous,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,sourcegraph.com
8,rec8VeVqBxJsUpNaD,2024-10-02T16:56:56.000Z,"{""name"":""ThousandEyes (part of Cisco)"",""descri...",ThousandEyes,http://www.thousandeyes.com,Approved,2024-10-02T16:56:56.000Z,2024-10-08T18:31:51.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,thousandeyes.com


In [3]:
sorted_companies = companies.sort_values(by="fields.Last Modified", ascending=False)
sorted_companies[["fields.Name", "fields.Last Modified"]].head(10)

,fields.Name,fields.Last Modified
33,The Coalition,2024-10-11T16:03:02.000Z
7,Sourcegraph,2024-10-10T15:02:54.000Z
48,Roam,2024-10-10T15:02:51.000Z
65,Sudowrite,2024-10-10T15:02:35.000Z
30,promptfoo,2024-10-10T15:02:34.000Z
81,Pave.dev,2024-10-10T15:02:31.000Z
74,Disney,2024-10-08T18:32:09.000Z
8,ThousandEyes,2024-10-08T18:31:51.000Z
47,Verkada Inc,2024-10-07T21:38:25.000Z
43,Hotel Engine,2024-10-07T21:37:55.000Z


In [4]:
company_objects = airtable.pandas_to_seeds(companies)
company_objects

3                         (TBD, TBD, tbd.website, None)
5              (Veritone, Veritone, veritone.com, None)
6     (DoubleDown Interactive, DoubleDown Interactiv...
7     (Sourcegraph, Sourcegraph, sourcegraph.com, None)
8     (ThousandEyes, ThousandEyes, thousandeyes.com,...
9     (Akili Interactive, EndeavorRx, akiliinteracti...
16    (Synthesize Bio, Synthesize Bio, synthesize.bi...
20             (98point6, 98point6, 98point6.com, None)
22    (Carbon Robotics, Laserweeder, carbonrobotics....
24    (Kevala, Kevala, kevala.care, (staffing, healt...
25       (Magnite, Magnite, magnite.com, (advertising))
28       (RescueTime, RescueTime, rescuetime.com, None)
29    (A Place for Mom, A Place for Mom, aplaceformo...
30          (promptfoo, promptfoo, promptfoo.dev, None)
31                         (Hiya, Hiya, hiya.com, None)
32        (Dolly, Dolly, dolly.com, (moving, delivery))
33    (The Coalition, Gears of War, thecoalitionstud...
34          (Clobotics, Clobotics, clobotics.com

In [5]:
force_rebuild_companies = {
    # "Sudowrite"
}

force_rebuild_objects = [company for company in company_objects if company.company in force_rebuild_companies]

assert len(force_rebuild_objects) == len(force_rebuild_companies), f"force_rebuild_objects: {force_rebuild_objects}"

In [6]:
from datetime import datetime, timedelta
import os
from typing import Optional

target = Seed.init("Imagine Pediatrics", domain="imaginepediatrics.org")
data_folder = "../output/data"

output_json = f"{data_folder}/{target.as_path_v2()}.json"
output_json

def get_file_age(file_path: str) -> Optional[timedelta]:
    if os.path.exists(file_path):
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        return datetime.now() - file_mod_time
    else:
        return None
    
def should_rebuild(target: Seed, file_path: str, max_age: timedelta = timedelta(days=7)) -> bool:
    if target.company in force_rebuild_companies:
        return True
    
    age = get_file_age(file_path)
    if age is None:
        return True
    return age > max_age

should_rebuild(target, output_json)


False

In [7]:
from loguru import logger

import unified

for target in company_objects:
    output_json = f"{data_folder}/{target.as_path_v2()}.json"

    if not should_rebuild(target, output_json):
        logger.info(f"Skipping {output_json} as it is up to date.")
    else:
        logger.info(f"Building {output_json}...")

        try:
            unified_result = await unified.run(
                target, 
                # TODO: Allow some customization of these parameters
                num_reddit_threads=10, 
                max_glassdoor_review_pages=5, 
                max_glassdoor_job_pages=0,
                max_news_articles=20,
                )
            
            with open(output_json, 'w') as json_file:
                json_file.write(unified_result.model_dump_json(indent=2))
        except IndexError as e:
            logger.error(f"Error, skipping {target.company}: {e}")



2024-10-11 09:03:27.391 | INFO     | __main__:<module>:9 - Skipping ../output/data/TBD.json as it is up to date.
2024-10-11 09:03:27.392 | INFO     | __main__:<module>:9 - Skipping ../output/data/Veritone.json as it is up to date.
2024-10-11 09:03:27.393 | INFO     | __main__:<module>:9 - Skipping ../output/data/DoubleDown_Interactive.json as it is up to date.
2024-10-11 09:03:27.393 | INFO     | __main__:<module>:9 - Skipping ../output/data/Sourcegraph.json as it is up to date.
2024-10-11 09:03:27.394 | INFO     | __main__:<module>:9 - Skipping ../output/data/ThousandEyes.json as it is up to date.
2024-10-11 09:03:27.395 | INFO     | __main__:<module>:9 - Skipping ../output/data/Akili_Interactive_EndeavorRx.json as it is up to date.
2024-10-11 09:03:27.395 | INFO     | __main__:<module>:9 - Skipping ../output/data/Synthesize_Bio.json as it is up to date.
2024-10-11 09:03:27.396 | INFO     | __main__:<module>:9 - Skipping ../output/data/98point6.json as it is up to date.
2024-10-11 09: